[Reference](https://medium.com/data-folks-indonesia/speed-up-pandas-performance-9cdd0290bee3)

In [1]:
import pandas as pd
import numpy as np
import numba

In [2]:
X = np.random.rand(100000, 5)
d_X = pd.DataFrame(X, columns=[f'c_{i}' for i in range(5)])

In [3]:
roll = d_X.rolling(5)

In [4]:
def moving_avg(x):
    return x.mean()

In [5]:
%timeit -n 1 -r 1 roll.apply(moving_avg)

1min 3s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%timeit -n 1 -r 1 roll.apply(moving_avg, raw=True)

4.25 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%timeit -n 1 -r 1 roll.apply(moving_avg, engine='numba', raw=True)

/usr/local/lib/python3.10/dist-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


1.88 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%timeit -n 1 -r 1 roll.apply(moving_avg, engine='cython')

56.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
def softmax(logit):
    e_x = np.exp(logit)
    result = e_x / e_x.sum()
    return result.argmax()

%timeit -n 1 -r 1 d_X.apply(softmax, axis=1)

%timeit -n 1 -r 1 d_X.apply(softmax, axis=1, raw=True)

@numba.njit
def softmax(logit):
    e_x = np.exp(logit)
    result = e_x / e_x.sum()
    return result.argmax()

@numba.njit
def apply_softmax(arr):
    labels = []
    for row in arr:
        label = softmax(row)
        labels.append(label)

    return labels

%timeit -n 1 -r 1 apply_softmax(d_X.to_numpy())

42.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
765 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
